In [2]:
import numpy as np
import pandas as pd 
import string
import re

In [3]:
with open('all_inovices.txt', mode='r', encoding='utf8', errors='ignore') as f:
    text = f.read()

In [4]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [5]:
df = pd.DataFrame(data[1:], columns=data[0])

In [6]:
df.head()

,id,text,tag,
0,001.jpeg,Invoice,O,
1,001.jpeg,no:,O,
2,001.jpeg,51109338,B-ID,
3,001.jpeg,Date,O,
4,001.jpeg,of,O,


In [7]:
whitespace =string.whitespace
punctuation = '!"#$%&\'()*+-.:;<=>?@[\\]^_`{|}~'
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removeWhitespace = text.translate(tableWhitespace)
    removePunctuation = removeWhitespace.translate(tablePunctuation)

    return str(removePunctuation)

In [8]:
df['text'] = df['text'].apply(cleanText)

In [9]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

In [10]:
dataClean.head(10)

,id,text,tag,
0,001.jpeg,invoice,O,
1,001.jpeg,no,O,
2,001.jpeg,51109338,B-ID,
3,001.jpeg,date,O,
4,001.jpeg,of,O,
5,001.jpeg,issue,O,
6,001.jpeg,seller,O,
7,001.jpeg,"andrews,",B-SN,
8,001.jpeg,kirby,I-SN,
9,001.jpeg,and,I-SN,


In [11]:
group = dataClean.groupby(by='id')

In [22]:
bills = group.groups.keys()

In [24]:
all_bills_data = []
for bill in bills :
    bill_data = []
    groupArray = group.get_group(bill)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in groupArray :
        text = str(text)
        stringLength = len(text)+1
    
        start = end
        end = start + stringLength
    
        if label != 'O':
            annot = (start, end-1, label)
            annotations['entities'].append(annot)
    
        content = content + text + ' '

    bill_data = (content, annotations)
    all_bills_data.append(bill_data)

In [33]:
all_bills_data[:5]

[('invoice no 91781923 date of issue 3/9/2013 seller client barryroberts lopez plc 9074 davis place 7417 bowers fort ellisfort, il 4641 longberg, tx 60292 tax id 912798265 tax id 934872950 iban gb22saze28844860501717 items no description qty um net price net worth vat tks gaming computer 4,00 each 620,00 2 480,00 10 2 hp pro 6200 desktop tower 1,00 each 179,95 179,95 10 computer core i5 31ghz 8gb 500gb windows 10 home wifi sentry gaming computer 5,00 each 594,99 2 974,95 10 ryzen 3 3200g 16 gb ram buildtoorder summary vat net worth 10 5 634,90 total 5 634,90 vat 563,49 563,49 gross worth 2 728,00 197,95 3 272,45 gross worth 6 198,39 6 198,39 ',
  {'entities': [(11, 19, 'B-ID'),
    (34, 42, 'B-DATE'),
    (57, 69, 'B-SN'),
    (70, 75, 'B-CN'),
    (76, 79, 'I-CN'),
    (191, 213, 'B-IBAN'),
    (638, 639, 'B-TOTAL'),
    (640, 646, 'I-TOTAL')]}),
 ('invoice no 12459784 date of issue seller smith, johnson and ramsey 3581 ruben gateway suite 160 new kayla, nh 28229 tax id 902964332 iban

In [26]:
import random

In [27]:
random.shuffle(all_bills_data)

In [28]:
len(all_bills_data)

300

In [29]:
train_data = all_bills_data[:265]
test_data =all_bills_data[265:]

In [30]:
import pickle

In [32]:
pickle.dump(train_data, open('./data/TrainData.pickle', mode='wb'))
pickle.dump(test_data, open('./data/TestData.pickle', mode='wb'))